# Implementing Different Layers

It is important to know how to implement different layers. In the prior recipe, we implemented fully connected layers. We will expand our knowledge of various layers in this recipe.

In [57]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

<br/>
<br/>
<hr/>
## One-dimensional data

In [58]:
ops.reset_default_graph()
sess = tf.Session()

### Inputs

In [59]:
DATA_SIZE = 25

In [60]:
x_vals = np.random.normal(size=DATA_SIZE)
print(x_vals[: 4])

[-1.6678073  -0.30906708  0.06410025  1.04731887]


In [61]:
x = tf.placeholder(dtype=tf.float32, shape=[DATA_SIZE], name='x')
print(x)

Tensor("x:0", shape=(25,), dtype=float32)


### Convolution layer

In [63]:
def convolution(input_1d, kernel, name='convolution_output'):
    with tf.name_scope('convolution'):
        # Expand 1d to 4d
        input_2d = tf.expand_dims(input_1d, axis=0, name='input_2d')
        input_3d = tf.expand_dims(input_2d, axis=0, name='input_3d')
        input_4d = tf.expand_dims(input_3d, axis=3, name='input-4d')

        print(input_2d)
        print(input_3d)
        print(input_4d)

        strides = [1, 1, 1, 1]
        padding = 'VALID'

        print('strides: {}'.format(strides))
        print('padding: {}'.format(padding))

        # Perform convolution
        conv = tf.nn.conv2d(input=input_4d, filter=kernel, strides=strides, padding=padding, name='conv')
        print(conv)

        # Collapse 4d to 1d
        conv_output = tf.squeeze(conv, name=name)
        return conv_output

_____
#### <u>FACT</u>: Calculating convolutional output size
_____

$$output\_size = (input\_size - filter\_size + 2 \times padding) / stride\_size + 1$$

For example:
* input of shape [2, 16, 18, 5]
  * samples = 2
  * height &times; width = [16, 18]
  * channels = 5
  
  
* filter of shape [6, 4, 5, 7]
  * height &times width = [6, 4]
  * input channels = 5
  * output channels = 7
  
  
* strides of shape [1, 3, 2, 1]
  * height &times width = [3, 2]
  
  
* Padding 'VALID'
  * padding = 0

$$output\_size = \left([16, 18] - [6, 4] + 2 \times 0\right) / [3, 2] + 1 = [4, 8]$$

* output will be of shape [2, 4, 8, 7]
  * samples = 2
  * height &times; width = [4, 8]
  * channels = 7

In [64]:
input_4d = tf.zeros(shape=[2, 16, 18, 5], name='input_4d')
filter_4d = tf.ones(shape=[6, 4, 5, 7], name='filter_4d')
strides = [1, 3, 2, 1]
padding = 'VALID'

output_4d = tf.nn.conv2d(input=input_4d, filter=filter_4d, strides=strides, padding=padding, name='output_4d')

print(input_4d)
print(filter_4d)
print(output_4d)

Tensor("input_4d:0", shape=(2, 16, 18, 5), dtype=float32)
Tensor("filter_4d:0", shape=(6, 4, 5, 7), dtype=float32)
Tensor("output_4d:0", shape=(2, 4, 8, 7), dtype=float32)


-----

In [65]:
kernel = tf.Variable(tf.random_normal(shape=[1, 5, 1, 1]), name='kernel')
print(kernel)

conv_output = convolution(x, kernel, 'conv_output')
print(conv_output)

Tensor("kernel/read:0", shape=(1, 5, 1, 1), dtype=float32)
Tensor("input_2d:0", shape=(1, 25), dtype=float32)
Tensor("input_3d:0", shape=(1, 1, 25), dtype=float32)
Tensor("input-4d:0", shape=(1, 1, 25, 1), dtype=float32)
strides: [1, 1, 1, 1]
padding: VALID
Tensor("conv:0", shape=(1, 1, 21, 1), dtype=float32)
Tensor("conv_output:0", shape=(21,), dtype=float32)


### Activation layer

In [66]:
def activation(input_1d, name='activation_output'):
    with tf.name_scope('activation'):
        return tf.nn.relu(input_1d, name=name)

In [67]:
act_output = activation(conv_output, 'act_output')
print(act_output)

Tensor("act_output:0", shape=(21,), dtype=float32)


### Pool layer

In [68]:
def pool(input_1d, width, name='pool_output'):
    with tf.name_scope('pool'):
        # Expand 1d to 4d
        input_2d = tf.expand_dims(input_1d, axis=0, name='input_2d')
        input_3d = tf.expand_dims(input_2d, axis=0, name='input_3d')
        input_4d = tf.expand_dims(input_3d, axis=3, name='input_4d')

        print(input_2d)
        print(input_3d)
        print(input_4d)

        ksize = [1, 1, width, 1]
        strides = [1, 1, 1, 1]
        padding = 'VALID'

        print('kside: {}'.format(ksize))
        print('strides: {}'.format(strides))
        print('padding: {}'.format(padding))

        # Perform max pool
        pool = tf.nn.max_pool(input_4d, ksize=ksize, strides=strides, padding=padding, name='pool')
        print(pool)

        # Collapse 4d to 1d
        pool_output = tf.squeeze(pool, name=name)
        return pool_output

_____
#### <u>FACT</u>: Calculating pool output size
_____

$$output\_size = (input\_size - filter\_size + 2 \times padding) / stride\_size + 1$$

For example:
* input of shape [2, 16, 18, 5]
  * samples = 2
  * height &times; width &times; channels = [16, 18, 5]
  
  
* window shape [1, 4, 5, 3]
  * height &times; width &times; channels = [4, 5, 3]
  
  
* strides [1, 3, 2, 2]
  * height &times; width &times; channels = [3, 2, 2]
  
  
* Padding 'VALID'
  * padding = 0

$$output\_size = \left([16, 18, 5] - [4, 5, 3] + 2 \times 0\right) / [3, 2, 2] + 1 = [5, 7, 2]$$

* output will be of shape [2, 5, 7, 2]
  * samples = 2
  * height &times; width = [5, 7]
  * channels = 2

In [69]:
max_pool_in = tf.zeros(shape=[2, 16, 18, 5], name='max_pool_in')
kside = [1, 4, 5, 3]
strides = [1, 3, 2, 2]
padding = 'VALID'

max_pool_out = tf.nn.max_pool(value=input_4d, ksize=kside, strides=strides, padding=padding, name='max_pool_out')

print(max_pool_in)
print(max_pool_out)

Tensor("max_pool_in:0", shape=(2, 16, 18, 5), dtype=float32)
Tensor("max_pool_out:0", shape=(2, 5, 7, 2), dtype=float32)


-----

In [70]:
pool_output = pool(act_output, 5, 'pool_output')
print(pool_output)

Tensor("input_2d_1:0", shape=(1, 21), dtype=float32)
Tensor("input_3d_1:0", shape=(1, 1, 21), dtype=float32)
Tensor("input_4d_1:0", shape=(1, 1, 21, 1), dtype=float32)
kside: [1, 1, 5, 1]
strides: [1, 1, 1, 1]
padding: VALID
Tensor("pool:0", shape=(1, 1, 17, 1), dtype=float32)
Tensor("pool_output:0", shape=(17,), dtype=float32)


### Fully-connected layer

In [102]:
def fully_connected(input_layer, num_outputs, name='fully_connected_output'):
    with tf.name_scope('fully_connected'):
        # expand 1d to 2d
        input_2d = tf.expand_dims(input_layer, 0, name='input_2d')
        print(input_2d)  

        # create weight and bias
        height = tf.shape(input_layer)
        width = [num_outputs]
        shape = tf.squeeze(tf.stack([height, width]))
        weight = tf.random_normal(shape=shape, stddev=0.1, name='weight')
        bias = tf.random_normal(shape=[num_outputs], name='bias')

        print(weight)
        print(bias)

        # calculate full layer
        full = tf.add(tf.matmul(input_2d, weight), bias, name='full')
        print(full)

        # collapse 2d to 1d
        full_output = tf.squeeze(full, name=name)
        return full_output

In [104]:
full_output = fully_connected(pool_output, 5, 'full_output')
print(full_output)

Tensor("fully_connected_1/input_2d:0", shape=(1, 17), dtype=float32)
Tensor("fully_connected_1/weight:0", shape=(?, ?), dtype=float32)
Tensor("fully_connected_1/bias:0", shape=(5,), dtype=float32)
Tensor("fully_connected_1/full:0", shape=(1, 5), dtype=float32)
Tensor("fully_connected_1/full_output:0", shape=(5,), dtype=float32)


### Initializing variables

In [105]:
init = tf.global_variables_initializer()
sess.run(init)

### Running graph and printing outputs

In [106]:
feed_dict = {x: x_vals}

conv_res, act_res, pool_res, full_res = sess.run([conv_output, act_output, pool_output, full_output], feed_dict=feed_dict)
print('Convolution:\n{}\n'.format(conv_res))
print('Activation:\n{}\n'.format(act_res))
print('Pool:\n{}\n'.format(pool_res))
print('Fully-connected:\n{}\n'.format(full_res))

Convolution:
[ 0.11242437 -1.11597371 -3.53005576  1.59111547  2.64025736  3.65019417
 -1.76622796 -1.0412904   1.14337957  0.3814013   3.0025847  -3.27095366
  1.32398546  3.46479893 -1.06887472  1.37958634 -4.85182524  2.58537221
 -0.46636114  2.5465951   1.98529458]

Activation:
[ 0.11242437  0.          0.          1.59111547  2.64025736  3.65019417
  0.          0.          1.14337957  0.3814013   3.0025847   0.
  1.32398546  3.46479893  0.          1.37958634  0.          2.58537221
  0.          2.5465951   1.98529458]

Pool:
[ 2.64025736  3.65019417  3.65019417  3.65019417  3.65019417  3.65019417
  3.0025847   3.0025847   3.0025847   3.46479893  3.46479893  3.46479893
  3.46479893  3.46479893  2.58537221  2.58537221  2.58537221]

Fully-connected:
[-0.13378757 -1.61461425  1.03143084 -1.9784348  -0.15626031]



<br/>
<br/>
<hr/>
## Two-dimensional data

In [125]:
ops.reset_default_graph()
sess = tf.Session()

### Inputs

In [126]:
N_ROWS = 10
N_COLS = 8
DATA_SIZE = [N_ROWS, N_COLS]
KERNEL_SIZE = 2
KERNEL_STRIDE_SIZE = 2
POOL_SIZE = 2
POOL_STRIDE_SIZE = 2

In [127]:
x_vals = np.random.normal(size=DATA_SIZE)
print(x_vals[: 2, : 4])

[[ 0.52201842  0.33799355 -0.48635399  0.23903511]
 [ 0.92599632  0.70205369  0.6411765  -0.85311417]]


In [128]:
x = tf.placeholder(dtype=tf.float32, shape=DATA_SIZE, name='x')
print(x)

Tensor("x:0", shape=(10, 8), dtype=float32)


### Convolution layer

In [129]:
def convolution(input_2d, kernel, name='convolution_output'):
    with tf.name_scope('convolution'):
        # Expand 2d to 4d
        input_3d = tf.expand_dims(input_2d, axis=0, name='input_3d')
        input_4d = tf.expand_dims(input_3d, axis=3, name='input-4d')

        print(input_3d)
        print(input_4d)

        strides = [1, KERNEL_STRIDE_SIZE, KERNEL_STRIDE_SIZE, 1]
        padding = 'VALID'

        print('strides: {}'.format(strides))
        print('padding: {}'.format(padding))

        # Perform convolution
        conv = tf.nn.conv2d(input=input_4d, filter=kernel, strides=strides, padding=padding, name='conv')
        print(conv)

        # Collapse 4d to 2d
        conv_output = tf.squeeze(conv, name=name)
        return conv_output

In [130]:
kernel = tf.Variable(tf.random_normal(shape=[KERNEL_SIZE, KERNEL_SIZE, 1, 1]), name='kernel')
print(kernel)

conv_output = convolution(x, kernel, 'conv_output')
print(conv_output)

Tensor("kernel/read:0", shape=(2, 2, 1, 1), dtype=float32)
Tensor("convolution/input_3d:0", shape=(1, 10, 8), dtype=float32)
Tensor("convolution/input-4d:0", shape=(1, 10, 8, 1), dtype=float32)
strides: [1, 2, 2, 1]
padding: VALID
Tensor("convolution/conv:0", shape=(1, 5, 4, 1), dtype=float32)
Tensor("convolution/conv_output:0", shape=(5, 4), dtype=float32)


### Activation layer

In [131]:
def activation(input_2d, name='activation_output'):
    with tf.name_scope('activation'):
        return tf.nn.relu(input_2d, name=name)

In [132]:
act_output = activation(conv_output, 'act_output')
print(act_output)

Tensor("activation/act_output:0", shape=(5, 4), dtype=float32)


### Pool layer

In [133]:
def pool(input_2d, height, width, stride, name='pool_output'):
    with tf.name_scope('pool'):
        # Expand 2d to 4d
        input_3d = tf.expand_dims(input_2d, axis=0, name='input_3d')
        input_4d = tf.expand_dims(input_3d, axis=3, name='input_4d')

        print(input_3d)
        print(input_4d)

        ksize = [1, height, width, 1]
        strides = [1, stride, stride, 1]
        padding = 'VALID'

        print('kside: {}'.format(ksize))
        print('strides: {}'.format(strides))
        print('padding: {}'.format(padding))

        # Perform max pool
        pool = tf.nn.max_pool(input_4d, ksize=ksize, strides=strides, padding=padding, name='pool')
        print(pool)

        # Collapse 4d to 2d
        pool_output = tf.squeeze(pool, name=name)
        return pool_output

In [134]:
pool_output = pool(act_output, POOL_SIZE, POOL_SIZE, POOL_STRIDE_SIZE, 'pool_output')
print(pool_output)

Tensor("pool/input_3d:0", shape=(1, 5, 4), dtype=float32)
Tensor("pool/input_4d:0", shape=(1, 5, 4, 1), dtype=float32)
kside: [1, 2, 2, 1]
strides: [1, 2, 2, 1]
padding: VALID
Tensor("pool/pool:0", shape=(1, 2, 2, 1), dtype=float32)
Tensor("pool/pool_output:0", shape=(2, 2), dtype=float32)


### Fully-connected layer

In [135]:
def fully_connected(input_layer, num_outputs, name='fully_connected_output'):
    with tf.name_scope('fully_connected'):
        # flatten input
        input_flat = tf.reshape(input_layer, shape=[-1], name='input_flat')
        print(input_flat)
        
        # create weight and bias
        height = tf.shape(input_flat)
        width = [num_outputs]
        shape = tf.squeeze(tf.stack([height, width]))
        weight = tf.random_normal(shape=shape, stddev=0.1, name='weight')
        bias = tf.random_normal(shape=[num_outputs], name='bias')

        print(weight)
        print(bias)

        # expand 1d to 2d
        input_2d = tf.expand_dims(input_flat, 0, name='input_2d')
        print(input_2d)
        
        # calculate full layer
        full = tf.add(tf.matmul(input_2d, weight), bias, name='full')
        print(full)

        # collapse 2d to 1d
        full_output = tf.squeeze(full, name=name)
        return full_output

In [136]:
full_output = fully_connected(pool_output, 5, 'full_output')
print(full_output)

Tensor("fully_connected/input_flat:0", shape=(4,), dtype=float32)
Tensor("fully_connected/weight:0", shape=(?, ?), dtype=float32)
Tensor("fully_connected/bias:0", shape=(5,), dtype=float32)
Tensor("fully_connected/input_2d:0", shape=(1, 4), dtype=float32)
Tensor("fully_connected/full:0", shape=(1, 5), dtype=float32)
Tensor("fully_connected/full_output:0", shape=(5,), dtype=float32)


### Initializing variables

In [137]:
init = tf.global_variables_initializer()
sess.run(init)

### Running graph and printing outputs

In [138]:
feed_dict = {x: x_vals}

conv_res, act_res, pool_res, full_res = sess.run([conv_output, act_output, pool_output, full_output], feed_dict=feed_dict)
print('Convolution:\n{}\n'.format(conv_res))
print('Activation:\n{}\n'.format(act_res))
print('Pool:\n{}\n'.format(pool_res))
print('Fully-connected:\n{}\n'.format(full_res))

Convolution:
[[ 1.89023542 -0.62432337 -0.45539266 -0.88576597]
 [-0.28317535 -0.85009599  0.22505143  0.99821234]
 [-0.55481005  2.22789359 -0.56118977 -1.43089116]
 [ 0.33216459 -2.14857769  1.09039211 -0.08050856]
 [ 0.32793519  2.27612281 -1.4064281  -1.04818475]]

Activation:
[[ 1.89023542  0.          0.          0.        ]
 [ 0.          0.          0.22505143  0.99821234]
 [ 0.          2.22789359  0.          0.        ]
 [ 0.33216459  0.          1.09039211  0.        ]
 [ 0.32793519  2.27612281  0.          0.        ]]

Pool:
[[ 1.89023542  0.99821234]
 [ 2.22789359  1.09039211]]

Fully-connected:
[ 0.30451173 -0.59224355  0.1254805  -0.15607134 -0.99479735]

